<a href="https://colab.research.google.com/github/amberxuqianchen/nlp-workshop/blob/main/NLP_Workshop_1_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data from online sources: 
- scrape it or 
- through Application Programming Interface (API)

# Scrape PBS news webpage

Before you do it, check robot.txt:
Simply type in your root domain, then add /robots. txt to the end of the URL.

In our case, https://psych.ucsb.edu/robots.txt

## 1st attempt
prompt: write a python code to scrape content from https://psych.ucsb.edu/news-events/news, I want the news title, dates, url and the news content

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://psych.ucsb.edu/news-events/news'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

news_items = soup.find_all('div', class_='views-row')

for item in news_items:
    title = item.find('div', class_='views-field-title').find('a').text.strip()
    date = item.find('div', class_='views-field-created').text.strip()
    news_url = item.find('div', class_='views-field-title').find('a')['href']
    
    # Request news article page
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.content, 'html.parser')
    
    # Extract news content
    content = news_soup.find('div', class_='field--name-body').text.strip()
    
    print(f"Title: {title}")
    print(f"Date: {date}")
    print(f"URL: {news_url}")
    print(f"Content: {content}")
    print()


## 2nd attempt
prompt 1/2: 
the date, link and url is like this:
<div class="all-news-wrap">
<div class="fancy-date"><time datetime="00Z">Jan</time>
<time datetime="00Z">25</time>
<time datetime="00Z">2023</time>
 </div>
<div>
<h3><a href="/news-events/news/2023/mentorship-program-access-grads-launched-uc-davis" hreflang="en">Mentorship Program Access Grads Launched At UC Davis</a></h3><p></p><p><span><span><span><span><span><span>Access Grads, a program pioneered by… <a href="/news-events/news/2023/mentorship-program-access-grads-launched-uc-davis" class="views-more-link">read more</a></span></span></span></span></span></span></p></div>
</div>

prompt 2/2:
in the news_url, I want the news content from this:

<div class="news__body"><p><span><span><span><span><span><span>Access Grads, a program pioneered by graduate students in PBS to promote mentoring of undergraduates aimed at helping them become involved in research and successfully navigate the process of applying for graduate school, </span></span></span></span></span></span><span><span><span><span><span><span><span>is pleased to announce a new chapter of their organization at </span></span></span></span></span></span></span><a href="https://ucdaccessgrads.wixsite.com/my-site"><span><span><span><span><span><span><span><span><span>UC Davis</span></span></span></span></span></span></span></span></span></a><span><span><span><span><span><span>! In the future, Access Grads hopes to expand to several other institutions, with the long-term goal of increasing diversity in science, and demystifying the process of applying to graduate school.</span></span></span></span></span></span></p>

<p><span><span><span><span><span><span>Learn more about the UCSB Access Grads program at here at </span></span></span></span></span></span><a href="https://accessgrads.wixsite.com/psychology"><span><span><span><span><span><span><span><span>UCSB</span></span></span></span></span></span></span></span></a><span><span><span><span><span><span> and at </span></span></span></span></span></span><a href="https://ucdaccessgrads.wixsite.com/my-site"><span><span><span><span><span><span><span><span><span>UC Davis</span></span></span></span></span></span></span></span></span></a><span><span><span><span><span><span>.</span></span></span></span></span></span></p></div>

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://psych.ucsb.edu/news-events/news'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

news_items = soup.find_all('div', class_='all-news-wrap')

for item in news_items:
    date = item.find('div', class_='fancy-date').get_text(separator=' ').strip()
    link = item.find('h3').find('a')
    title = link.text.strip()
    news_url = link['href']
    
    # Request news article page
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.content, 'html.parser')
    
    # Extract news content
    content = news_soup.find('div', class_='field--name-body').text.strip()
    
    print(f"Title: {title}")
    print(f"Date: {date}")
    print(f"URL: {news_url}")
    print(f"Content: {content}")
    print()


MissingSchema: ignored

In [ ]:
news_url
# https://psych.ucsb.edu/news-events/news/2023/mentorship-program-access-grads-launched-uc-davis

'/news-events/news/2023/pbs-student-elle-murata-give-tedx-talk'

## 3rd attempt: success!
prompt: this one works: news_url = f"https://psych.ucsb.edu{news_url}", modify the code

or tell chatGPT to keep the main domain: "https://psych.ucsb.edu/"

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://psych.ucsb.edu/news-events/news'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

news_items = soup.find_all('div', class_='all-news-wrap')

for item in news_items:
    date = item.find('div', class_='fancy-date').get_text(separator=' ').strip()
    link = item.find('h3').find('a')
    title = link.text.strip()
    news_url = link['href']
    
    # Check if URL has a scheme and add one if missing
    if not news_url.startswith('http'):
        news_url = f"https://psych.ucsb.edu{news_url}"
    
    # Request news article page
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.content, 'html.parser')
    
    # Extract news content
    content = news_soup.find('div', class_='news__body').text.strip()
    
    print(f"Title: {title}")
    print(f"Date: {date}")
    print(f"URL: {news_url}")
    print(f"Content: {content}")
    print()


Title: PBS Student Elle Murata To Give TEDx Talk
Date: Jan 
 25 
 2023
URL: https://psych.ucsb.edu/news-events/news/2023/pbs-student-elle-murata-give-tedx-talk
Content: PBS graduate student Elle Murata will give a talk at the 2023 TEDxLagunaBlanca speaker series on February 1. The theme is Metamorphosis. 
In her talk, "When the Science Becomes Personal: Tales from a Patient-Researcher", Murata will share her dual experience as a woman in science and a woman underserved by the biomedical sciences. Elle Murata is a third year PhD candidate in Cognition, Perception and Cognitive  Neuroscience at PBS, where her research examines the impact of sex hormones on the brain. In the Jacobs lab, Elle leverages neuroimaging to interrogate how menopause and reproductive disorders shape neural structure and function. 
For more information about this exciting event, see here: https://www.tedxlagunablancaschool.com/copy-of-beneath-the-surface-2022

Title: Mentorship Program Access Grads Launched At UC 

## plus save the data!

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup

url = 'https://psych.ucsb.edu/news-events/news'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

news_items = soup.find_all('div', class_='all-news-wrap')

with open('psych_news.csv', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Date', 'URL', 'text'])
    
    for item in news_items:
        date = item.find('div', class_='fancy-date').get_text(separator=' ').strip()
        link = item.find('h3').find('a')
        title = link.text.strip()
        news_url = link['href']
        
        # Check if URL has a scheme and add one if missing
        if not news_url.startswith('http'):
            news_url = f"https://psych.ucsb.edu{news_url}"
        
        # Request news article page
        news_response = requests.get(news_url)
        news_soup = BeautifulSoup(news_response.content, 'html.parser')
        
        # Extract news content
        content = news_soup.find('div', class_='news__body').text.strip()
        
        writer.writerow([title, date, news_url, content])


In [ ]:
import pandas as pd

df = pd.read_csv('psych_news.csv')
df

,Title,Date,URL,text
0,PBS Student Elle Murata To Give TEDx Talk,Jan \n 25 \n 2023,https://psych.ucsb.edu/news-events/news/2023/p...,PBS graduate student Elle Murata will give a t...
1,Mentorship Program Access Grads Launched At UC...,Jan \n 25 \n 2023,https://psych.ucsb.edu/news-events/news/2023/m...,"Access Grads, a program pioneered by graduate ..."
2,Prof. Emily Jacobs to lead new Ann S. Bowers W...,Jan \n 17 \n 2023,https://psych.ucsb.edu/news-events/news/2023/p...,"Associate Professor Emily Jacobs, whose labora..."
3,Professor Heejung Kim Receives Outstanding Con...,Jan \n 10 \n 2023,https://psych.ucsb.edu/news-events/news/2023/p...,PBS Professor Heejung Kim has received the 202...
4,PBS Statement on UAW Strike,Nov \n 18 \n 2022,https://psych.ucsb.edu/news-events/news/2022/p...,The PBS faculty acknowledge the pressures on o...
5,Emeritus Professor Gerald Jacobs receives Life...,Nov \n 2 \n 2022,https://psych.ucsb.edu/news-events/news/2022/e...,Jerry Jacobs has received the 2022 Richard C. ...
6,PBS Faculty Emily Jacobs Named Top 10 Scientis...,Oct \n 17 \n 2022,https://psych.ucsb.edu/news-events/news/2022/p...,Associate Professor Emily Jacobs was named a T...
7,PBS Professor Michael Beyeler Receives $1.5 Mi...,Oct \n 4 \n 2022,https://psych.ucsb.edu/news-events/news/2022/p...,"Good vision is essential for everyday life, bu..."
8,PBS Chair and graduate student Elizabeth Quinn...,Sep \n 28 \n 2022,https://psych.ucsb.edu/news-events/news/2022/p...,"Professor Shelly Gable, the PBS chair, togethe..."
9,PBS Faculty Richard Mayer: The most productive...,Sep \n 16 \n 2022,https://psych.ucsb.edu/news-events/news/2022/p...,Distinguished UCSB Professor in Psychological ...


⚡ This code only scrapes the first page of the news as demonstration. Always cross-check the data completeness before next step.

# Reddit (API)

For example, scrape subreddit 'confession' to study guilt.

follow the tutorial: https://towardsdatascience.com/scraping-reddit-data-1c0af3040768

# Twitter (API)
Application and tutorial: https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/2-choosing-the-right-product-track.md